# Arquitecturas Especializadas

### Profesor: Juan Mario Haut Hurtado
### 4º Ingeniería en Informática en Ingeniería de Computadores
__________________________________________________________________

# Variables

In [1]:
import torch
from torch.autograd import Variable

In [2]:
# creamos un tensor
tensor = torch.FloatTensor([[1,2],[3,4]])

# creamos una variable de Torch, normalemnte requieren computar gradientes
variable = Variable(tensor, requires_grad=True)

print(tensor)       # [torch.FloatTensor of size 2x2]
print(variable)     # [torch.FloatTensor of size 2x2]

tensor([[1., 2.],
        [3., 4.]])
tensor([[1., 2.],
        [3., 4.]], requires_grad=True)


El tensor y la variable parecen iguales.

Sin embargo, la variable con autograd forma parte parte del grafo de ejecución y por tanto, es parte del gradiente automático. 

In [3]:
t_out = torch.mean(tensor*tensor)       # x^2
v_out = torch.mean(variable*variable)   # x^2
print(t_out)
print(v_out)

tensor(7.5000)
tensor(7.5000, grad_fn=<MeanBackward0>)


In [4]:
v_out.backward()    # backpropagation from v_out

$$ v_{out} = {{1} \over {4}} sum(variable^2) $$

los gradientes respecto a las variables, 

$$ {d(v_{out}) \over d(variable)} = {{1} \over {4}} 2 variable = {variable \over 2}$$

Comprobemos el resultado:

In [5]:
variable.grad

tensor([[0.5000, 1.0000],
        [1.5000, 2.0000]])

In [6]:
variable # estos son datos en formato variable 

tensor([[1., 2.],
        [3., 4.]], requires_grad=True)

In [7]:
variable.data # estos son datos en formato tensor

tensor([[1., 2.],
        [3., 4.]])

In [8]:
variable.data.numpy() # estos son datos en formato numpy

array([[1., 2.],
       [3., 4.]], dtype=float32)

Ojo, nosotros hicimos `.backward()` en `v_out` pero `variable` se le han asignado nuevos valores en `grad`.

Como esta linea
```
v_out = torch.mean(variable*variable)
``` 
Hará una nueva variable `v_out` y la conectará con `variable` en el grafo de computación.

In [9]:
type(v_out)

torch.Tensor

In [10]:
type(v_out.data)

torch.Tensor

Para pasar la variable de cpu a cuda o de cuda a cpu, símplmente llamamos a la función

In [11]:
variableCUDA = variable.cuda()
print(variableCUDA)
variableCPU = variableCUDA.cpu()
print(variableCPU)

tensor([[1., 2.],
        [3., 4.]], device='cuda:0', grad_fn=<ToCopyBackward0>)
tensor([[1., 2.],
        [3., 4.]], grad_fn=<ToCopyBackward0>)


In [12]:
# No puedo operar convariables en diferentes dispositivos
resultadoFinal = variableCUDA + variableCPU
print(resultadoFinal)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [13]:
# Deben estar alojados en el mismo sitio
resultadoFinal = variableCUDA + variableCPU.cuda()
print("CUDA", resultadoFinal)

# Deben estar alojados en el mismo sitio
resultadoFinal = variableCUDA.cpu() + variableCPU
print("CPU", resultadoFinal)

CUDA tensor([[2., 4.],
        [6., 8.]], device='cuda:0', grad_fn=<AddBackward0>)
CPU tensor([[2., 4.],
        [6., 8.]], grad_fn=<AddBackward0>)
